In [73]:
"""初始化"""

import pandas as pd
import numpy as np

stocks = pd.read_csv('csv/stocks-cny.csv', dtype={'Symbol':str})
trades = pd.read_csv('csv/trades-cny.csv', dtype={'Symbol':str, 'Qty':np.int64})
dividends = pd.read_csv('csv/dividends-cny.csv', dtype={'Symbol':str, 'Qty':np.int64})
stocks.set_index('Symbol', inplace=True)
trades.set_index('Symbol', inplace=True)
dividends.set_index('Symbol', inplace=True)

In [77]:
def get_trade_indicators(stocks, trades, dividends):
    '''
    stocks: A pandas dataframe that lists all traded stocks.
    trades: A pandas dataframe that lists all trades.
    dividends: A pandas dataframe that lists all dividends.

    Return a dataframe that includes all useful trade indicators.
    For example, Total buy/sell amount for a stock and average buy/sell price.
    '''

    # Calculate actual amount for each trade.
    proceed = trades['Price'] * trades['Qty'].abs()
    fee = trades['Commission'] + trades['Tax']
    trades['Basis'] = np.where( trades['Transaction']=='BUY',
            proceed+fee, proceed-fee)

    # Generate Pandas groupby object for all traded stocks.
    group_trade = trades.groupby([trades.index, trades['Transaction']])

    # Claculate total bought/sold qty for each stock.
    qty_sum = group_trade['Qty'].sum().unstack(fill_value=0)
    stocks_detail = stocks.join(qty_sum)
    stocks_detail.columns = ['Name', 'B_Qty', 'S_Qty']

    # Calculate average bought/sold price.
    basis = group_trade['Basis'].sum().unstack(fill_value=0)
    stocks_detail['B_Cost'] = basis['BUY']/stocks_detail['B_Qty']
    stocks_detail['S_Cost'] = basis['SELL']/stocks_detail['S_Qty'].abs()


    return stocks_detail

In [78]:
"""
为每笔交易计算实际金额

买入交易：买入价格 * 买入股数 + 佣金 + 税
卖出交易：卖出价格 * 卖出股数 - 佣金 - 税
"""

proceed = trades['Price'] * trades['Qty'].abs()
fee = trades['Commission'] + trades['Tax']
trades['Basis'] = np.where(trades['Transaction']=='BUY', proceed+fee, proceed-fee)
group_trade = trades.groupby([trades.index, trades['Transaction']])

In [80]:
"""测试"""

stocks_detail = get_trade_indicators(stocks, trades, dividends)
stocks_detail

,Symbol,Name,B_Qty,S_Qty,B_Cost,S_Cost
0,510900,H股ETF,140000,-140000,0.970250,0.866763
1,150176,H股B,218300,-218300,0.709829,0.698363
2,159920,恒生ETF,180000,-180000,1.116000,1.122129
3,600519,贵州茅台,650,-650,215.153077,312.227154
4,600036,招商银行,8000,-8000,17.990000,16.117931
5,600660,福耀玻璃,7000,-4000,17.223781,20.509110
6,600886,国投电力,22000,-22000,6.815591,6.561269
7,000651,格力电器,7000,-7000,21.424286,23.213803
8,000002,万科A,3000,-3000,24.430000,19.076127
9,002780,三夫户外,500,-500,41.560000,85.311220


In [5]:
"""为每只股票计算总买入额，总卖出额，平均买入价格，平均卖出价格和已实现盈亏"""

buyorsell_basis = group_trade['Basis'].sum().unstack()
stocks_hkd_summary['Buy_Basis'] = buyorsell_basis['BUY']
stocks_hkd_summary['Sell_Basis'] = buyorsell_basis['SELL']
stocks_hkd_summary['Buy_Cost'] = stocks_hkd_summary['Buy_Basis']/stocks_hkd_summary['Buy_Qty']
stocks_hkd_summary['Sell_Cost'] = stocks_hkd_summary['Sell_Basis']/stocks_hkd_summary['Sell_Qty'].abs()
stocks_hkd_summary['Realized_PnL'] = stocks_hkd_summary['Sell_Basis'] - stocks_hkd_summary['Buy_Cost']*stocks_hkd_summary['Sell_Qty'].abs()

In [6]:
"""为每只股票计算累计分红"""

dividends_hkd['Dividend'] = dividends_hkd['PerShare']*dividends_hkd['Qty']-dividends_hkd['Commission']-dividends_hkd['Tax']

#对所有的分红记录按股票分组
group_dividend = dividends_hkd.groupby(dividends_hkd.index)

#stock_dividend = group_dividend['Dividend'].sum()
stocks_hkd_summary['Dividend'] = group_dividend['Dividend'].sum()


In [7]:
"""对于已实现盈亏和分红用零值替换NaN"""

stocks_hkd_summary['Realized_PnL'].fillna(0.0, inplace=True)
stocks_hkd_summary['Dividend'].fillna(0.0, inplace=True)

In [8]:
"""定义获取股票最新报价的函数"""

import datetime
import pandas_datareader.data as web
import tushare as ts

def getYahooQuote(symbols):
    start = datetime.datetime.today()
    end = start
    
    quotes_dict = {}
    yahoo_symbols = symbols + '.HK'
    pnl = web.DataReader(yahoo_symbols, 'yahoo', start, end)
    df = pnl['Close'].T
    df.index.name = 'Symbol'
    df.columns=['Last']
    df.index = df.index.map(lambda x: x.rstrip('.HK'))
    
    return df


def getTuShareQuote(symbols):
    cons = ts.get_apis()
    last = datetime.datetime.today() - datetime.timedelta(days=7)
    
    quotes_dict = {}
    tushare_symbols = '0' + symbols
    for symbl in tushare_symbols:
        df = ts.bar(symbl, conn=cons, asset='X', start_date=last, end_date='')
        quotes_dict[symbl[1:]] = df['close'].iloc[0]

    quotes = pd.Series(quotes_dict)
    return quotes

In [9]:
"""为当前持仓股票获取最近价格"""

stocks_hkd_hold = stocks_hkd_summary.loc[lambda df: df.Qty > 0]
stock_last = getTuShareQuote(stocks_hkd_hold.index)
stocks_hkd_summary['Last'] = stock_last

In [10]:
"""为未平仓股票计算未实现盈亏"""

stocks_hkd_summary['Unrealized_PnL'] = stocks_hkd_summary['Qty'] * (stocks_hkd_summary['Last'] - stocks_hkd_summary['Buy_Cost'])
#对未已实现盈亏和分红用零值替换NaN
stocks_hkd_summary['Unrealized_PnL'].fillna(0.0, inplace=True)
stocks_hkd_summary['Earning'] = stocks_hkd_summary['Dividend'] + stocks_hkd_summary['Realized_PnL'] + stocks_hkd_summary['Unrealized_PnL']

In [12]:
'''为每只股票生成最终盈亏总结'''

stocks_final = stocks_hkd_summary[['Name', 'Last', 'Qty', 'Dividend', 'Realized_PnL', 'Earning']]
stocks_final.loc[stocks_final['Qty']>0].round(2)

,Name,Last,Qty,Dividend,Realized_PnL,Earning
Symbol,,,,,,
1988,民生银行,7.42,12000,1733.87,2834.05,-5004.67
0512,远大医药,2.94,32000,0.00,-9048.17,27115.54
1378,中国宏桥,12.16,10000,0.00,-1356.19,50455.61
1918,融创中国,35.00,10000,24771.23,975918.32,1268304.57
2601,中国太保,38.05,6000,0.00,2533.30,22265.40
2666,环球医疗,8.02,20000,0.00,0.00,26194.48
0392,北京控股,45.50,5000,2078.50,0.00,26167.80
0656,复星国际,18.88,8000,0.00,410.63,2323.25
0735,中国电力新能源,4.75,35000,0.00,0.00,2109.12
